In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd
import tensorflow as tf
import nltk

import numpy as np

import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling1D, Embedding

In [0]:
dataset_filename = "../raw/cyber-trolls.csv"

In [0]:
df = pd.read_csv(dataset_filename, header=None)
df.head()

,0,1
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook R...,1
2,why did you fuck it up. I could do it all day ...,1
3,Dude they dont finish enclosing the fucking sh...,1
4,WTF are you talking about Men? No men thats no...,1


# Preproceessing Part

In [0]:
texts = np.asarray(df[0])
labels = np.asarray(df[1])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size = 0.20)

In [0]:
vocab_size=10000

def norm(text):
  return one_hot(text, vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')

X_train = [norm(x) for x in X_train]
X_test = [norm(x) for x in X_test]

In [0]:
max_length = 256
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')


# Modeling

In [0]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 32, input_shape=(None,)))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(X_train,
          y_train,
          epochs=100,
          batch_size=512,
          verbose=1)

Epoch 1/100
16000/16000 [==============================] - 1s 50us/step - loss: 0.3444 - acc: 0.8530
Epoch 2/100
16000/16000 [==============================] - 1s 49us/step - loss: 0.3390 - acc: 0.8567
Epoch 3/100
16000/16000 [==============================] - 1s 48us/step - loss: 0.3359 - acc: 0.8563
Epoch 4/100
16000/16000 [==============================] - 1s 50us/step - loss: 0.3325 - acc: 0.8611
Epoch 5/100
16000/16000 [==============================] - 1s 49us/step - loss: 0.3302 - acc: 0.8599
Epoch 6/100
16000/16000 [==============================] - 1s 49us/step - loss: 0.3262 - acc: 0.8631
Epoch 7/100
16000/16000 [==============================] - 1s 51us/step - loss: 0.3227 - acc: 0.8639
Epoch 8/100
16000/16000 [==============================] - 1s 48us/step - loss: 0.3205 - acc: 0.8653
Epoch 9/100
16000/16000 [==============================] - 1s 49us/step - loss: 0.3174 - acc: 0.8666
Epoch 10/100
16000/16000 [==============================] - 1s 51us/step - loss: 0.3146 - a

In [0]:
results = model.evaluate(X_test, y_test)

results

4001/4001 [==============================] - 0s 45us/step


[0.45798601133767974, 0.8222944263934017]

In [0]:
len(X_train)

16000

# Amazon Review Data에 Evaluate

In [0]:
review = pd.read_csv("/content/gdrive/My Drive/data/angelcs/amazon_reviews.csv")
review.head()

In [0]:
review_x = np.asarray(review['reviews.text'], dtype='object')
review_x = [norm(str(x)) for x in review_x]
review_x = pad_sequences(review_x, maxlen=max_length, padding='pre')

In [0]:
p = model.predict(review_x)

In [0]:
p_result = [(review.loc[i, 'reviews.text'], prob[0]) for i, prob in enumerate(p)]
  

In [0]:
p_result_df = pd.DataFrame(p_result, columns=["Text","Prob"])
p_result_df.head()

,Text,Prob
0,This product so far has not disappointed. My c...,2.845225e-02
1,great for beginner or experienced person. Boug...,5.737841e-04
2,Inexpensive tablet for him to use and learn on...,5.173683e-05
3,I've had my Fire HD 8 two weeks now and I love...,0.000000e+00
4,I bought this for my grand daughter when she c...,4.768372e-07


In [0]:
p_result_df.to_csv("/content/gdrive/My Drive/data/angelcs/amazon_reviews_prediction.csv")